## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-27-01-16-11 +0000,nypost,Dr. Oz promises to ‘aggressively’ go after ‘ma...,https://nypost.com/2026/01/26/us-news/dr-oz-pr...
1,2026-01-27-01-12-24 +0000,nyt,Trump Says He Will Raise Tariffs on South Kore...,https://www.nytimes.com/2026/01/26/business/ec...
2,2026-01-27-01-12-20 +0000,nypost,Dem lawmaker makes ‘solidarity’ trip to Minnea...,https://nypost.com/2026/01/26/us-news/dem-lawm...
3,2026-01-27-01-12-19 +0000,bbc,'Major step': French MPs vote in favour of bil...,https://www.bbc.com/news/articles/c07x003vx0yo...
4,2026-01-27-01-10-58 +0000,startribune,Border czar Tom Homan’s arrival in Minnesota s...,https://www.startribune.com/tom-homan-minnesot...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2450/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,49
24,minneapolis,23
46,minnesota,20
345,shooting,16
57,ice,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
264,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...,127
243,2026-01-26-14-24-51 +0000,nypost,Trump sending border czar Tom Homan to Minneso...,https://nypost.com/2026/01/26/us-news/trump-se...,112
62,2026-01-26-23-04-08 +0000,wapo,President Donald Trump said Monday that he spo...,https://www.washingtonpost.com,106
228,2026-01-26-15-14-23 +0000,bbc,Trump says administration 'reviewing everythin...,https://www.bbc.com/news/articles/cr571qg4m61o...,105
201,2026-01-26-16-56-15 +0000,latimes,Scrutiny builds over killing of Alex Pretti; T...,https://www.latimes.com/world-nation/story/202...,105


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
264,127,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...
70,85,2026-01-26-22-52-00 +0000,wsj,Democrats want changes to the Department of Ho...,https://www.wsj.com/politics/policy/government...
243,63,2026-01-26-14-24-51 +0000,nypost,Trump sending border czar Tom Homan to Minneso...,https://nypost.com/2026/01/26/us-news/trump-se...
145,61,2026-01-26-19-56-06 +0000,wapo,Six presumed dead in private jet crash at Main...,https://www.washingtonpost.com/transportation/...
20,55,2026-01-27-00-42-00 +0000,wsj,The Army will gain access to Salesforce’s tech...,https://www.wsj.com/politics/national-security...
73,51,2026-01-26-22-48-07 +0000,nypost,"Shackled, greasy-haired, disgraced Olympian Ry...",https://nypost.com/2026/01/26/us-news/former-o...
157,48,2026-01-26-19-10-29 +0000,nypost,Sen. Adam Schiff threatens government shutdown...,https://nypost.com/2026/01/26/us-news/adam-sch...
244,41,2026-01-26-14-19-04 +0000,nypost,"Minnesota CEOs, sports teams demand ‘immediate...",https://nypost.com/2026/01/26/business/minneso...
210,38,2026-01-26-16-07-27 +0000,nypost,Stock market grew in 2025 — but less than any ...,https://nypost.com/2026/01/26/business/stock-m...
235,35,2026-01-26-14-57-07 +0000,cbc,Israel recovers remains of final hostage in Ga...,https://www.cbc.ca/news/world/gaza-israel-rema...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
